In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [ ]:
import numpy as np
import os
import gzip

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
from sklearn.model_selection import train_test_split
# 注意housing本身的转变 以及train_test_split参数设置
x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state=7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [ ]:
# 1、手动设置超参数集合  但是每个epoch中lr还是不会变
# 缺点：1）参数多循环嵌套多。 2）无法并行处理
learning_rates = [1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2]
histories = []
for lr in learning_rates:
    model = keras.models.Sequential([
        keras.layers.Dense(30, activation='relu', input_shape=x_train.shape[1:]),
        keras.layers.Dense(1)
    ])
    
    # 创建optimizer
    optimizer = keras.optimizers.SGD(lr)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]
    history = model.fit(x_train_scaled, y_train, validation_data=(x_valid_scaled, y_valid),
                        epochs=10, callbacks=callbacks)
    
    # 创建列表 保存fit的history
    histories.append(history)

In [ ]:
def draw_learning_curve(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid()
    plt.ylim(0, 1)
    plt.show()
    
for lr, history in zip(learning_rates, histories):
    print("learning_rate:\t", lr)
    draw_learning_curve(history)

In [33]:
# 2、使用sklearn  没有evaluate函数
# step1：转化为sklearn的model
# step2：定义参数集合
# step3：搜索参数

def build_model(hidden_layers = 1,
                layer_size = 30,
                learning_rate = 3e-3):
    model = keras.models.Sequential()
    # 独立出来，因为有输入层
    model.add(keras.layers.Dense(layer_size, activation='relu', input_shape=x_train.shape[1:]))
    for i in range(hidden_layers -1):
        model.add(keras.layers.Dense(layer_size, activation='relu'))
    
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

# step1  注意（）中是函数而不是实例
sklearn_model = keras.wrappers.scikit_learn.KerasRegressor(build_model)
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]
'''sklearn_history = sklearn_model.fit(x_train_scaled, y_train, validation_data=(x_valid_scaled, y_valid),
                            epochs=20, callbacks=callbacks)
draw_learning_curve(sklearn_history)
'''

# step 2  参数集合就是build_model中的形参
from scipy.stats import reciprocal
# f(x) = 1 / (x*long(b/a))  a <= x <= b  概率分布
# 字典
param_distribution = {
    "hidden_layers": [1, 2, 3, 4],
    "layer_size": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2)
}
from sklearn.model_selection import RandomizedSearchCV
random_search_cv = RandomizedSearchCV(sklearn_model, param_distribution,
                                     n_iter=10, n_jobs=1)
random_search_cv.fit(x_train_scaled, y_train, epochs=1)

TypeError: Singleton array array(1) cannot be considered a valid collection.